In [5]:
from data_loader import data_loader
from RNN_model import RNN_model

import torch.nn as nn


sample_num = 16 * 12
target_num = 3
max_seq_len = 1104


input_dim = 255 # Fake.

hidden_dim = 30
layer_dim = 2
output_dim = 4  # Four kinds of genres within 12 songs.


In [6]:
seqs, labels = data_loader('music-affect_v2', sample_num, max_seq_len, target_num)

TypeError: can't assign a numpy.ndarray to a torch.FloatTensor

In [ ]:
print(seqs.shape())
print(seqs)
print(labels)
    

In [ ]:

lengths = 1104
rnn = RNN_model(input_dim, hidden_dim, layer_dim, output_dim)

x = nn.utils.rnn.pack_padded_sequence(data, lengths, batch_first=False)
x, h = rnn(x)
x, lengths = nn.utils.rnn.pad_packed_sequence(x, batch_first=False)



In [ ]:

print(rnn)


print(rnn.rnn.all_weights)
print("+++++++++++++++++++++++++++++++++++++")
print(rnn.fc.weight)

